In [2]:
import pandas as pd

raw_data = pd.read_csv(
    "/data/hetianran/didi/chengdu/gps_20161101",
    names=["driver_id", "order_id", "timestamp", "lon", "lat"],
)

In [3]:
raw_data.head()

,driver_id,order_id,timestamp,lon,lat
0,co85eh7fc1wr8jxsioej4e7ga9Et.nCq,7ig5behe5_xw8kqB7ed84mlma9zv4jxx,1477969147,104.07513,30.72724
1,co85eh7fc1wr8jxsioej4e7ga9Et.nCq,7ig5behe5_xw8kqB7ed84mlma9zv4jxx,1477969150,104.07513,30.72702
2,co85eh7fc1wr8jxsioej4e7ga9Et.nCq,7ig5behe5_xw8kqB7ed84mlma9zv4jxx,1477969154,104.07504,30.72672
3,co85eh7fc1wr8jxsioej4e7ga9Et.nCq,7ig5behe5_xw8kqB7ed84mlma9zv4jxx,1477969156,104.07497,30.72630
4,co85eh7fc1wr8jxsioej4e7ga9Et.nCq,7ig5behe5_xw8kqB7ed84mlma9zv4jxx,1477969159,104.07497,30.72582


In [18]:
raw_data['traj_id'], unique_ids = pd.factorize(raw_data['order_id'])
raw_data

,driver_id,order_id,timestamp,lon,lat,traj_id
0,co85eh7fc1wr8jxsioej4e7ga9Et.nCq,7ig5behe5_xw8kqB7ed84mlma9zv4jxx,1477969147,104.07513,30.72724,0
1,co85eh7fc1wr8jxsioej4e7ga9Et.nCq,7ig5behe5_xw8kqB7ed84mlma9zv4jxx,1477969150,104.07513,30.72702,0
2,co85eh7fc1wr8jxsioej4e7ga9Et.nCq,7ig5behe5_xw8kqB7ed84mlma9zv4jxx,1477969154,104.07504,30.72672,0
3,co85eh7fc1wr8jxsioej4e7ga9Et.nCq,7ig5behe5_xw8kqB7ed84mlma9zv4jxx,1477969156,104.07497,30.72630,0
4,co85eh7fc1wr8jxsioej4e7ga9Et.nCq,7ig5behe5_xw8kqB7ed84mlma9zv4jxx,1477969159,104.07497,30.72582,0
...,...,...,...,...,...,...
32155512,jaci6eh96_qwahqz9bbd8bfmh5qs3lCA,f9k5c99eh-wx0fvm4f65ghkea6rB7pun,1477976392,104.10102,30.67833,181171
32155513,jaci6eh96_qwahqz9bbd8bfmh5qs3lCA,f9k5c99eh-wx0fvm4f65ghkea6rB7pun,1477976393,104.10101,30.67833,181171
32155514,jaci6eh96_qwahqz9bbd8bfmh5qs3lCA,f9k5c99eh-wx0fvm4f65ghkea6rB7pun,1477976394,104.10100,30.67834,181171
32155515,jaci6eh96_qwahqz9bbd8bfmh5qs3lCA,f9k5c99eh-wx0fvm4f65ghkea6rB7pun,1477976396,104.10100,30.67834,181171


In [24]:
from trajlib.data_processing.utils.data_definition import TrajectoryData
from tqdm import tqdm


def load_data_chengdu(raw_data) -> TrajectoryData:
    traj_data = TrajectoryData()

    extracted_df = raw_data.loc[:, ["traj_id", "driver_id", "order_id"]]
    extracted_df = extracted_df.drop_duplicates()
    extracted_df = extracted_df.reset_index(drop=True)

    traj_data.traj_table = extracted_df

    data_1 = []
    data_2 = []
    point_counter = 1
    for traj_id, timestamp, lon, lat in tqdm(
        zip(
            raw_data["traj_id"],
            raw_data["timestamp"],
            raw_data["lon"],
            raw_data["lat"],
        )
    ):
        data_1.append(
            {
                "point_id": point_counter,
                "timestamp": timestamp,
                "traj_id": traj_id,
            }
        )
        data_2.append({"lon": lon, "lat": lat})
        point_counter += 1

    traj_data.batch_append_point_data(
        new_point_data_list=data_1, extra_attr_list=data_2
    )

    return traj_data


traj_data = load_data_chengdu(raw_data)

32155517it [00:40, 791695.00it/s]
/data/niuyiming/TrajMM/trajlib/data_processing/utils/data_definition.py:96: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat([self.point_table, new_row_df], ignore_index=True)


In [25]:
traj_data.point_table.head()

,point_id,traj_id,timestamp,lon,lat
0,1,0,1477969147,104.07513,30.72724
1,2,0,1477969150,104.07513,30.72702
2,3,0,1477969154,104.07504,30.72672
3,4,0,1477969156,104.07497,30.72630
4,5,0,1477969159,104.07497,30.72582
